In [1]:
import os
os.chdir('..')

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [3]:
from src.api.common.dependency_container import DependencyContainer
from IPython.display import HTML  

DC = DependencyContainer(); DC.initialize()
pregunta='Whatsapp, necesito asesor por whatsapp, dame el número'
request = {'Pregunta': pregunta, 'FAQs': faqs + faqs_unir}
response = DependencyContainer.get_schema_generator_workflow().execute(request)
HTML(f"<h2>{response['FAQ correspondiente']}</h2>"+response['Contenido'])